In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
customer_df = pd.read_csv('/kaggle/input/datasets/archissave/client/client_data.csv')
price_df = pd.read_csv('/kaggle/input/datasets/archissave/prices/price_data.csv')

Drop identifier columns

In [3]:
customer_df = customer_df.drop(columns=['id'])

Remove low-information or redundant columns

In [4]:
customer_df.nunique().sort_values()

has_gas                               2
churn                                 2
origin_up                             6
channel_sales                         8
nb_prod_act                          10
forecast_discount_energy             12
num_years_antig                      13
forecast_price_pow_off_peak          41
forecast_price_energy_peak          329
date_end                            368
date_renewal                        386
forecast_price_energy_off_peak      516
pow_max                             698
date_activ                         1796
cons_gas_12m                       2112
date_modif_prod                    2129
margin_net_pow_ele                 2391
margin_gross_pow_ele               2391
forecast_meter_rent_12m            3528
forecast_cons_year                 4218
cons_last_month                    4751
imp_cons                           7752
cons_12m                          11065
net_margin                        11965
forecast_cons_12m                 13993


Convert date columns

In [5]:
date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
for col in date_cols:
    customer_df[col] = pd.to_datetime(customer_df[col])

Create tenure-related features

In [6]:
customer_df['customer_tenure_days'] = (
    customer_df['date_end'] - customer_df['date_activ']
).dt.days

In [7]:
customer_df['years_since_activation'] = (
    pd.Timestamp.today() - customer_df['date_activ']
).dt.days / 365

Renewal timing features

In [8]:
customer_df['days_until_renewal'] = (
    customer_df['date_renewal'] - pd.Timestamp.today()
).dt.days

Consumption stability

In [9]:
customer_df['consumption_ratio'] = (
    customer_df['cons_last_month'] / (customer_df['cons_12m'] / 12)
)

Forecast vs actual consumption

In [10]:
customer_df['forecast_error'] = (
    customer_df['forecast_cons_12m'] - customer_df['cons_12m']
)

Profitability indicators

In [11]:
customer_df['margin_ratio'] = (
    customer_df['margin_net_pow_ele'] / customer_df['net_margin']
)

Aggregate pricing data per customer

In [12]:
price_agg = price_df.groupby('id').agg(
    avg_off_peak_var=('price_off_peak_var', 'mean'),
    max_off_peak_var=('price_off_peak_var', 'max'),
    min_off_peak_var=('price_off_peak_var', 'min'),
    price_volatility=('price_off_peak_var', 'std')
).reset_index()

Price change magnitude

In [13]:
price_agg['price_range'] = (
    price_agg['max_off_peak_var'] - price_agg['min_off_peak_var']
)

Combine Datasets

In [18]:
print(customer_df.columns)

Index(['channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'churn',
       'customer_tenure_days', 'years_since_activation', 'days_until_renewal',
       'consumption_ratio', 'forecast_error', 'margin_ratio'],
      dtype='object')


In [19]:
print(price_agg.columns)

Index(['id', 'avg_off_peak_var', 'max_off_peak_var', 'min_off_peak_var',
       'price_volatility', 'price_range'],
      dtype='object')


In [22]:
final_df = price_agg.merge(price_agg, on='id', how='left')

In [23]:
final_df = final_df.drop(columns=['id'])

In [24]:
final_df.info()
final_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16096 entries, 0 to 16095
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   avg_off_peak_var_x  16096 non-null  float64
 1   max_off_peak_var_x  16096 non-null  float64
 2   min_off_peak_var_x  16096 non-null  float64
 3   price_volatility_x  16096 non-null  float64
 4   price_range_x       16096 non-null  float64
 5   avg_off_peak_var_y  16096 non-null  float64
 6   max_off_peak_var_y  16096 non-null  float64
 7   min_off_peak_var_y  16096 non-null  float64
 8   price_volatility_y  16096 non-null  float64
 9   price_range_y       16096 non-null  float64
dtypes: float64(10)
memory usage: 1.2 MB


,avg_off_peak_var_x,max_off_peak_var_x,min_off_peak_var_x,price_volatility_x,price_range_x,avg_off_peak_var_y,max_off_peak_var_y,min_off_peak_var_y,price_volatility_y,price_range_y
count,16096.000000,16096.000000,16096.000000,16096.000000,16096.000000,16096.000000,16096.000000,16096.000000,16096.000000,16096.000000
mean,0.141022,0.145316,0.135414,0.004234,0.009902,0.141022,0.145316,0.135414,0.004234,0.009902
std,0.024184,0.025228,0.024344,0.005247,0.014042,0.024184,0.025228,0.024344,0.005247,0.014042
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.124068,0.128067,0.118238,0.002185,0.004462,0.124068,0.128067,0.118238,0.002185,0.004462
50%,0.147251,0.149555,0.143943,0.003220,0.007434,0.147251,0.149555,0.143943,0.003220,0.007434
75%,0.150171,0.153048,0.147600,0.004310,0.008979,0.150171,0.153048,0.147600,0.004310,0.008979
max,0.278759,0.280700,0.275253,0.068978,0.236095,0.278759,0.280700,0.275253,0.068978,0.236095
